In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_groq import ChatGroq


from dotenv import load_dotenv
import os

load_dotenv()


True

In [2]:

loader = PyPDFLoader(
    "https://images.colombo.com.br/produtos/902211/902211_1534441611766.pdf"
)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents_splits = text_splitter.split_documents(documents)
documents_splits

[Document(metadata={'source': 'https://images.colombo.com.br/produtos/902211/902211_1534441611766.pdf', 'page': 0}, page_content='1REFRIGERADOR TF39, TF39S MANUAL DE INSTRUÇÕES'),
 Document(metadata={'source': 'https://images.colombo.com.br/produtos/902211/902211_1534441611766.pdf', 'page': 1}, page_content='2Neste manual, você encontra todas as informações necessárias \npara sua segurança e uso adequado do seu refrigerador. Leia-o \npor inteiro antes da instalação e uso do eletrodoméstico e guarde \nesse guia para futuras consultas.\nEm caso de qualquer dúvida, ligue para o Serviço de Atendimento \nao Consumidor (3004 8778 para capitais e regiões metropolitanas ou \n0800 728 8778 para demais localidades).\nGuarde a nota fiscal de compra. A garantia só é válida mediante sua apre -\nsentação no Serviço Autorizado Electrolux.\nA etiqueta de identificação onde constam informações importantes como có -\ndigo, modelo e fabricante, entre outras, será utilizada pelo Serviço Autorizado \nElect

In [3]:
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")) 
# embeddings = OllamaEmbeddings(model='llama3')
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x74d879d3ff40>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x74d879aee650>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [4]:
db = FAISS.from_documents(documents_splits, embedding=embeddings)
db

In [6]:
# embeddings = OllamaEmbeddings(model='mxbai-embed-large')


# query = "O que não posso deixar sobre o refrigerador?"
query = "o que a garantia não cobre?"

documents_similarity = db.similarity_search(query)

# documents_similarity

page_contents = [doc.page_content.replace("\n", "") for doc in documents_similarity]

all_contents = "".join(page_contents)

all_contents

prompt = f"""
	Você é um ótimo assistente, que responde tudo sobre o contexto abaixo
	{documents_similarity},
 	Responda: {query}
  """

# model = ChatOpenAI(model='gpt-3.5-turbo')
# model = ChatOllama(model="llama3")
model =  ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key=os.getenv('GROQ_API_KEY')
)


response = model.invoke(prompt)

response.content

'De acordo com o texto, a garantia não cobre:\n\n* Maus tratos, descuidos ou alterações feitas por pessoas ou entidades não credenciadas pela Electrolux do Brasil S.A. (item 10)\n* Defeitos causados por acidente ou má utilização do produto pelo Consumidor (item 11)\n* Despesas com instalação do produto (item 12)\n* Não funcionamento ou falhas decorrentes de problemas de fornecimento de energia elétrica e/ou falta de água no local onde o produto está instalado (item 13)\n* Despesas com transporte, peças, materiais e mão de obra para preparação do local onde será instalado o produto (item 14)\n* Chamadas relacionadas unicamente a orientação de uso constantes no Manual de Instruções ou no próprio produto (item 15)\n* Despesas de deslocamento do Serviço Autorizado para atendimento (item 16)\n* Produto instalado ou utilizado em desacordo com o Manual de Instruções e utilizado para fins diferentes do uso doméstico (item 8)\n* Produto ligado em tensão diferente a qual foi destinado (item 9)\n